# Alan Author Strikes Again: More on Confirming Conjunctions of Disconfirmed Hypotheses

This notebook corresponds to the following paper:

- https://academic.oup.com/analysis/advance-article-abstract/doi/10.1093/analys/anad103/7731114

## Importing Packages and Setup

In [43]:
import numpy as np
import pandas as pd
import polars as pl
from scipy import stats
from itertools import chain, combinations
from IPython.display import Math, display
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

## Defining Functions for Powerset and Probabilities

In [6]:
# Powerset function (without the empty set)
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1,len(s)+1))

# Function that calculates all marginal and joint probabilities for a set of probability distributions, given as matrix
def probabilities(matrix):

    # Size of matrix, number of random variables
    length, width = matrix.shape
    num_variables = np.log2(width).astype(int)
    
    # Create matrix with all Boolean combinations
    binmat = np.array([np.fromstring(np.binary_repr(i,num_variables), dtype=np.uint8)==49 for i in range(0,width)])

    # Pick all probabilities from matrix
    result = [pd.Series(np.sum(np.all(binmat[:,i],axis=1)*matrix,axis = 1), name = f"P{i}") for i in powerset(range(num_variables))]

    return pl.DataFrame(result)

## Generating probability functions

In [34]:
# Number of variables and distributions
number_variables = 3
number_distributions = 1000000

# Sampling from Dirichlet distribution, calculating relevant probabilities
matrix = pl.DataFrame(np.random.dirichlet(np.ones(2**number_variables), number_distributions))
df = probabilities(matrix)

## Results

In [46]:
# Conjunction is confirmed (condition 1)
conjunction_confirmation = (df["P(0, 1, 2)"] > df["P(0,)"]*df["P(1, 2)"])
conjunction_confirmation_mean = conjunction_confirmation.mean()

# Conjuncts are disconfirmed (condition 2)
conjuncts_disconfirmation = (df["P(0, 1)"] < df["P(0,)"]*df["P(1,)"]) & (df["P(0, 2)"] < df["P(0,)"]*df["P(2,)"])
conjuncts_disconfirmation_mean = conjuncts_disconfirmation.mean()

# Conjunction of negated conjuncts is confirmed (condition 3)
negated_conjunction_confirmation = df["P(0, 1)"] + df["P(0, 2)"] - df["P(0, 1, 2)"] < df["P(0,)"] * (df["P(1,)"] + df["P(2,)"] - df["P(1, 2)"] )
negated_conjunction_confirmation_mean = negated_conjunction_confirmation.mean()

# Alan Author Effect
alan_author_effect = (conjunction_confirmation & conjuncts_disconfirmation)
alan_author_effect_conjunctive_prevalence = alan_author_effect.mean()
alan_author_effect_conditional_prevalence = alan_author_effect_conjunctive_prevalence/conjuncts_disconfirmation_mean

# Strong Alan Author Effect (conjunctive)
strong_alan_author_effect = (conjunction_confirmation & conjuncts_disconfirmation & negated_conjunction_confirmation)
strong_alan_author_effect_conjunctive_prevalence = strong_alan_author_effect.mean()

# Strong Alan Author Effect (conditional)
antecedent_strong_alan_author_effect =  (conjuncts_disconfirmation & negated_conjunction_confirmation)
antecedent_strong_alan_author_effect_mean = antecedent_strong_alan_author_effect.mean()
strong_alan_author_effect_conditional_prevalence = strong_alan_author_effect_conjunctive_prevalence/antecedent_strong_alan_author_effect_mean

# Prevalence of the effects
alan_author_effect_conjunctive_prevalence
alan_author_effect_conditional_prevalence
strong_alan_author_effect_conjunctive_prevalence
strong_alan_author_effect_conditional_prevalence

# Collecting results

results = pd.DataFrame([
    [alan_author_effect_conjunctive_prevalence, alan_author_effect_conditional_prevalence],
    [strong_alan_author_effect_conjunctive_prevalence, strong_alan_author_effect_conditional_prevalence],
]).rename(columns={0: "Conjunctive Prevalence", 1: "Conditional Prevalence"})

effects = pd.DataFrame([
['Alan Author Effect'],
['Strong Alan Author Effect'],
]).rename(columns={0: "Effect"})

pd.concat([effects,results],axis=1).style.hide()

Effect,Conjunctive Prevalence,Conditional Prevalence
Alan Author Effect,0.025006,0.100333
Strong Alan Author Effect,0.025006,0.111471
